In [ ]:
from collections import deque   # deque nos permite implementar colas (FIFO)
import random   

# SISTEMA DE TIENDA ONLINE con PILA, COLA, BÚSQUEDA y TABLA HASH
# Simular el funcionamiento de una tienda online utilizando
# distintas estructuras de datos:
# - Tabla hash → para almacenar clientes y acceder rápido a ellos.
# - Búsqueda secuencial/binaria → para localizar productos.
# - Cola → para manejar pedidos en orden de llegada (FIFO).
# - Historial (pila) → para registrar acciones de cada cliente.



# Tabla Hash personalizada de clientes

class TablaHashClientes:
    def __init__(self, tamaño=10):
        """
        Constructor de la tabla hash de clientes
        """
        self.tamaño = tamaño
        self.tabla = [[] for _ in range(tamaño)]
        self.contador = 0

    def funcion_hash(self, clave):
        """
        Convierte una clave en un índice de bucket.
        """
        return sum(ord(c) for c in str(clave)) % self.tamaño

    def put(self, clave, valor):
        """
        Inserta o actualiza un cliente en la tabla hash.
        """
        indice = self.funcion_hash(clave)
        for i, (k, v) in enumerate(self.tabla[indice]):
            if k == clave:  
                self.tabla[indice][i] = (clave, valor)  
                return
        self.tabla[indice].append((clave, valor)) 
        self.contador += 1

    def get(self, clave):
        """
        Devuelve el valor a partir de la clave.
        """
        indice = self.funcion_hash(clave)
        for k, v in self.tabla[indice]:
            if k == clave:
                return v
        return None  

    def contains(self, clave):
        """Verifica si un cliente existe en la tabla."""
        return self.get(clave) is not None

    def __len__(self):
        """Devuelve cuántos clientes hay registrados en total."""
        return self.contador

    def __str__(self):
        """
        Muestra la tabla hash de forma entendible
        """
        resultado = []
        for i, bucket in enumerate(self.tabla):
            resultado.append(f"Bucket {i}: {len(bucket)} clientes")
            for cliente_id, datos in bucket:
                resultado.append(f"  - {cliente_id}: {datos['nombre']}")
        return "\n".join(resultado)

# 2. Sistema de búsqueda de productos

class SistemaBusqueda:
    def __init__(self):
        """
        Constructor define un catálogo de productos y sus precios.
        """
        self.productos = {
            "electronics": ["Laptop", "Smartphone", "Tablet", "Smartwatch", "Headphones"],
            "clothing": ["T-Shirt", "Jeans", "Jacket", "Dress", "Shoes"],
            "books": ["Novel", "Textbook", "Magazine", "Comic", "Biography"]
        }
        self.precios = {
            "Laptop": 899.99, "Smartphone": 599.99, "Tablet": 399.99,
            "Smartwatch": 199.99, "Headphones": 99.99, "T-Shirt": 19.99,
            "Jeans": 49.99, "Jacket": 79.99, "Dress": 59.99, "Shoes": 89.99,
            "Novel": 14.99, "Textbook": 89.99, "Magazine": 5.99,
            "Comic": 3.99, "Biography": 12.99
        }

    def buscar_producto(self, consulta):
        """
        Búsqueda por texto recorre todos los productos y busca coincidencias
        con la consulta 
        """
        resultados = []
        consulta = consulta.lower()

        for categoria, productos in self.productos.items():
            for producto in productos:
                if consulta in producto.lower():
                    resultados.append((producto, categoria, self.precios[producto]))

       
        return sorted(resultados, key=lambda x: x[0])

    def busqueda_binaria_precios(self, precio_min, precio_max):
        """
        Búsqueda por rango de precios utilizando búsqueda binaria.

        """
        productos_ordenados = sorted(self.precios.items(), key=lambda x: x[1])
        resultados = []

        izquierda, derecha = 0, len(productos_ordenados) - 1
        while izquierda <= derecha:  
           
            medio = (izquierda + derecha) // 2  
            
            producto, precio = productos_ordenados[medio]  
           
            if precio_min <= precio <= precio_max:  
                # CASO 1: El precio del producto está dentro del rango buscado.
        
                resultados.append((producto, precio))
    
                i = medio - 1
                while i >= 0 and productos_ordenados[i][1] >= precio_min:
                    
                    resultados.append(productos_ordenados[i])
                    i -= 1  
        
                i = medio + 1
                while i < len(productos_ordenados) and productos_ordenados[i][1] <= precio_max:
                    
                    resultados.append(productos_ordenados[i])
                    i += 1  
        
                break  
            elif precio < precio_min:  
                # CASO 2: El precio medio es demasiado bajo.
               
                izquierda = medio + 1
        
            else:  
                # CASO 3: El precio medio es demasiado alto.

                derecha = medio - 1
        
        
        return sorted(set(resultados), key=lambda x: x[1])

# 3. Cola de pedidos

class Pedidos:
    def __init__(self):
        """
        Implementa una cola FIFO de pedidos.
        """
        self.cola = deque()
        self.contador_pedidos = 0

    def nuevo_pedido(self, id_cliente, productos, total):
        """
        Crea un nuevo pedido y lo agrega a la cola.
        """
        self.contador_pedidos += 1
        pedido_id = f"P{self.contador_pedidos:04d}"  # ID con formato P0001
        pedido = {
            'id': pedido_id,
            'cliente': id_cliente,
            'productos': productos,
            'total': total,
            'estado': 'pendiente'
        }
        self.cola.append(pedido)
        print(f" Pedido {pedido_id} recibido de cliente {id_cliente}")
        return pedido_id

    def atender_pedido(self):
        """
        Atiende el primer pedido de la cola (FIFO).
        """
        if self.cola:
            pedido = self.cola.popleft()
            pedido['estado'] = 'completado'
            print(f"Pedido {pedido['id']} atendido - Total: ${pedido['total']:.2f}")
            return pedido
        return None

    def estado_cola(self):
        """Devuelve cuántos pedidos están en cola."""
        return f"Pedidos en cola: {len(self.cola)}"


# -------------------------
# 4. Sistema de historial
# -------------------------
class HistorialCliente:
    def __init__(self):
        """
        Representa el historial de un cliente:
        
        """
        self.acciones = []
        self.pedidos = []

    def agregar_accion(self, accion):
        """Agrega una acción al historial (ej. búsqueda o compra)."""
        self.acciones.append(accion)
        print(f"Registrada acción: {accion}")

    def agregar_pedido(self, pedido_info):
        """Agrega un pedido al historial del cliente."""
        self.pedidos.append(pedido_info)

    def obtener_ultimas_acciones(self, n=5):
        """Devuelve las últimas n acciones del cliente."""
        return self.acciones[-n:] if self.acciones else []

    def obtener_historial_pedidos(self):
        """Devuelve todos los pedidos hechos por el cliente."""
        return self.pedidos


# SIMULACIÓN DEL SISTEMA

if __name__ == "__main__":
    print("INICIANDO SISTEMA DE TIENDA ONLINE")
    print("=" * 50)

    # --- Sistema de búsqueda ---
    buscador = SistemaBusqueda()

    # --- Tabla hash de clientes ---
    clientes = TablaHashClientes()

    # Registrar algunos clientes en la tabla hash
    clientes_registrados = [
        ("C001", "Ana Pérez", "ana@email.com"),
        ("C002", "Luis Gómez", "luis@email.com"),
        ("C003", "María López", "maria@email.com"),
        ("C004", "Carlos Ruiz", "carlos@email.com")
    ]
    for cliente_id, nombre, email in clientes_registrados:
        clientes.put(cliente_id, {
            "nombre": nombre,
            "email": email,
            "historial": HistorialCliente()
        })

    # --- Cola de pedidos ---
    sistema_pedidos = Pedidos()

    # DEMOSTRACIÓN DE BÚSQUEDA
    print("\n DEMOSTRACIÓN DE BÚSQUEDA DE PRODUCTOS")
    print("-" * 40)

    print("Búsqueda de 'phone':")
    resultados = buscador.buscar_producto("phone")
    for producto, categoria, precio in resultados:
        print(f"  - {producto} ({categoria}): ${precio:.2f}")

    print("\nBúsqueda por rango de precios ($50 - $200):")
    resultados_rango = buscador.busqueda_binaria_precios(50, 200)
    for producto, precio in resultados_rango:
        print(f"  - {producto}: ${precio:.2f}")

    # DEMOSTRACIÓN DE TABLA HASH
    print("\n DEMOSTRACIÓN DE TABLA HASH DE CLIENTES")
    print("-" * 40)
    print("Estructura de la tabla hash:")
    print(clientes)
    print(f"\nTotal de clientes registrados: {len(clientes)}")

    # SIMULACIÓN DE COMPRAS
    print("\n  SIMULACIÓN DE COMPRAS")
    print("-" * 40)

    # Cliente 1 realiza búsqueda y compra
    cliente1 = clientes.get("C001")
    print(f"Cliente: {cliente1['nombre']}")
    cliente1['historial'].agregar_accion("Búsqueda: 'laptop'")
    resultados = buscador.buscar_producto("laptop")
    if resultados:
        producto, categoria, precio = resultados[0]
        pedido_id = sistema_pedidos.nuevo_pedido("C001", [producto], precio)
        cliente1['historial'].agregar_accion(f"Compra: {producto} por ${precio:.2f}")
        cliente1['historial'].agregar_pedido({'id': pedido_id, 'productos': [producto], 'total': precio})

    # Cliente 2 realiza múltiples búsquedas y compra múltiple
    cliente2 = clientes.get("C002")
    print(f"\nCliente: {cliente2['nombre']}")
    cliente2['historial'].agregar_accion("Búsqueda: 'shoes'")
    resultados_zapatos = buscador.buscar_producto("shoes")
    cliente2['historial'].agregar_accion("Búsqueda: 'jacket'")
    resultados_chaqueta = buscador.buscar_producto("jacket")
    if resultados_zapatos and resultados_chaqueta:
        zapatos = resultados_zapatos[0]
        chaqueta = resultados_chaqueta[0]
        total = zapatos[2] + chaqueta[2]
        pedido_id = sistema_pedidos.nuevo_pedido("C002", [zapatos[0], chaqueta[0]], total)
        cliente2['historial'].agregar_accion(f"Compra múltiple: ${total:.2f}")
        cliente2['historial'].agregar_pedido({'id': pedido_id, 'productos': [zapatos[0], chaqueta[0]], 'total': total})

    # ATENDER PEDIDOS
    print("\n ATENDIENDO PEDIDOS")
    print("-" * 40)
    print(sistema_pedidos.estado_cola())
    for _ in range(2):
        pedido_atendido = sistema_pedidos.atender_pedido()
        if pedido_atendido:
            cliente = clientes.get(pedido_atendido['cliente'])
            cliente['historial'].agregar_accion(f"Pedido {pedido_atendido['id']} completado")

    # HISTORIAL DE CLIENTES
    print("\n HISTORIAL DE CLIENTES")
    print("-" * 40)
    for cliente_id in ["C001", "C002"]:
        cliente = clientes.get(cliente_id)
        print(f"\nHistorial de {cliente['nombre']}:")
        for accion in cliente['historial'].obtener_ultimas_acciones():
            print(f"  • {accion}")

    # VERIFICACIÓN DE COLISIONES
    print("\n VERIFICACIÓN DE COLISIONES EN TABLA HASH")
    print("-" * 40)
    for i, bucket in enumerate(clientes.tabla):
        if len(bucket) > 1:  # si hay más de un cliente en el mismo bucket
            print(f"Bucket {i} tiene {len(bucket)} clientes (colisión):")
            for cliente_id, datos in bucket:
                print(f"  - {cliente_id}: {datos['nombre']}")

    print("\n SIMULACIÓN COMPLETADA")
